## Appel des bibliothèques

### Installation des bibliothèques

In [1]:
!pip install eurostat gdown

### Importation des bibliothèques

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import json
import eurostat

import gdown

In [3]:
pd.set_option("display.max_columns", None)

## Utilisation des données via API

### Liste des pays de l'UE :

**Variable :**  
code_EU  
**Source :**  
Ministère de l'Enseignement Supérieur et de la Recherche  (MESR)  
**Description :**  
Liste des pays de l'Union Européenne dans lesquels l'UE pourrait investir dans le cadre d'une harmonisation des politiques de transition énergétique

In [4]:
#Liste des correspondances entre les codes pays
url = "https://data.enseignementsup-recherche.gouv.fr/api/explore/v2.1/catalog/datasets/curiexplore-pays/records?select=name_en%2Cname_fr%2Ciso2%2Ciso3&where=ue27%20%3D%20%22True%22&limit=-1"
response = requests.get(url)
data = response.json()

# Transformer en DataFrame
code_EU = pd.DataFrame(data['results'])
code_EU

,name_en,name_fr,iso2,iso3
0,Croatia,Croatie,HR,HRV
1,Romania,Roumanie,RO,ROU
2,Latvia,Lettonie,LV,LVA
3,Austria,Autriche,AT,AUT
4,Lithuania,Lituanie,LT,LTU
5,Spain,Espagne,ES,ESP
6,Slovenia,Slovénie,SI,SVN
7,Belgium,Belgique,BE,BEL
8,Netherlands,Pays-Bas,NL,NLD
9,Poland,Pologne,PL,POL


### Données utilisées :

In [5]:
##########################################################################
#
#	Définition des variables :
#	Les variables sont issues de l'API d'Eurostat (organisme de statistiques officielles pour l'Union Européenne).
#	Ces donné et ont été importée grâce à la bibliothèque eurostat puis décrite brièvement.
#	En tant qu'institution européenne la DG Energy se base essentiellement sur des données propres publiées sur Eurostat en vue d'établir ses prévisions.
#
##########################################################################

**Variable :** code_EU  
**Source :** Eurostat  
**Description :** Prix de l'électricité pour les foyers européens en Total-kWh TTC, avec prise en compte de la parité de pouvoir d'achat entre les pays. Les relevés bi-mensuelle ont été moyennés  
**Last update :**
19/12/2024 23:00  
**URL :**
 https://ec.europa.eu/eurostat/databrowser/view/nrg_pc_204__custom_11690203/bookmark/table?lang=en&bookmarkId=d8c29aa7-b162-4f62-b026-e026eb7babec

#### Electricity prices for household consumers - bi-annual data

In [6]:
electricity_price = eurostat.get_data_df('nrg_pc_204')

# Renommer la colonne pour éviter les problèmes avec le backslash
electricity_price = electricity_price.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
electricity_price = electricity_price[electricity_price["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
electricity_price = electricity_price[electricity_price["nrg_cons"]=='TOT_KWH'] # TOT_KWH - consommation totale en kWh
electricity_price = electricity_price[electricity_price["tax"]=='X_TAX'] # [X_TAX] - Toutes taxes comprises (All taxes included)
electricity_price = electricity_price[electricity_price["currency"]=='PPS'] # PPS (Purchasing Power Standard) – Standard de pouvoir d'achat

#supprimer les colonnes inutiles
electricity_price = electricity_price.drop(columns=electricity_price.loc[:, "2007-S1":"2021-S1"].columns)
electricity_price = electricity_price.drop(columns=electricity_price.columns[0:electricity_price.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = electricity_price.columns.get_loc('country') #récupérer l'indice de la colonne country
df = electricity_price.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
electricity_price['mean_energy_price'] = df.mean(axis=1) #calcul la moyenne des colonnes
electricity_price = electricity_price.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
electricity_price.head()

,country,mean_energy_price
2043,AT,0.198467
2045,BE,0.257600
2046,BG,0.164633
2047,CY,0.226533
2048,CZ,0.263067


In [7]:
#regarder le pays manquants dans electricity_price

unique_col1 = set(electricity_price["country"]) - set(code_EU["iso2"])  # Présent dans col1 mais pas col2
unique_col2 = set(code_EU["iso2"]) - set(electricity_price["country"])  # Présent dans col2 mais pas col1

print("Uniquement dans electricity_price :", unique_col1)
print("Uniquement dans code_EU :", unique_col2) #il manque GR/Grèce

Uniquement dans electricity_price : set()
Uniquement dans code_EU : {'GR'}


La grèce est la donnée manquante dans les différents dataset disponible sur Eurostat

#### Share of energy from renewable sources

**Variable :** share_renew_energy  
**Source :** Eurostat  
**Description :** Part de production électrique issu des énergies renouvelables - en pourcentage de la production nationale   
**Last Update :** 21/01/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_REN__custom_2076933/bookmark/table?lang=en&bookmarkId=c19d2992-11d6-440f-b18a-107f950d8f43

In [8]:
share_renew_energy = eurostat.get_data_df('nrg_ind_ren')

# Renommer la colonne pour éviter les problèmes avec le backslash
share_renew_energy = share_renew_energy.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
share_renew_energy = share_renew_energy.rename(columns={"2023": "MixEn_2023"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
share_renew_energy = share_renew_energy[share_renew_energy["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
share_renew_energy = share_renew_energy[share_renew_energy["nrg_bal"]=='REN']

#supprimer les colonnes inutiles
share_renew_energy = share_renew_energy.drop(columns=share_renew_energy.columns[0:share_renew_energy.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = share_renew_energy.columns.get_loc('country') #récupérer l'indice de la colonne country
df = share_renew_energy.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
share_renew_energy['mean_share_renew_energy'] = df.mean(axis=1) #calcul la moyenne des colonnes
share_renew_energy = share_renew_energy.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
share_renew_energy.head()


,country,mean_share_renew_energy
1,AT,31.83285
3,BE,7.67355
4,BG,16.11650
5,CY,9.75290
6,CZ,12.94330


#### Real GDP per capita

**Variable :** GDP_per_capita  
**Source :** Eurostat  
**Description :** Produit intérieur brut par habitant en parité de pouvoir d'achat afin de déterminer le niveau de richesse de la population et le comparer au reste de l'UE  
**Last Update :** 21/01/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_REN__custom_2076933/bookmark/table?lang=en&bookmarkId=c19d2992-11d6-440f-b18a-107f950d8f43

In [9]:
GDP_per_capita = eurostat.get_data_df('sdg_08_10')

# Renommer la colonne pour éviter les problèmes avec le backslash
GDP_per_capita = GDP_per_capita.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
GDP_per_capita = GDP_per_capita.rename(columns={"2023": "GPDpH_2023"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GDP_per_capita = GDP_per_capita[GDP_per_capita["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
GDP_per_capita = GDP_per_capita[GDP_per_capita["unit"]=='CLV_PCH_PRE_HAB'] #en parité de pouvoir d'achat par habitatants

#supprimer les colonnes inutiles
GDP_per_capita = GDP_per_capita.drop(columns=GDP_per_capita.columns[0:GDP_per_capita.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GDP_per_capita.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GDP_per_capita.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GDP_per_capita['mean_GDP_per_capita'] = df.mean(axis=1) #calcul la moyenne des colonnes
GDP_per_capita = GDP_per_capita.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GDP_per_capita.head()



,country,mean_GDP_per_capita
38,AT,0.916667
39,BE,1.100000
40,BG,4.383333
42,CY,1.825000
43,CZ,2.229167


#### HICP - inflation rate

**Variable :** HICP_inflation  
**Source :** Eurostat  
**Description :** Mesure de l'inflation sur la base des indices des prix à la consommation harmonisé afin de comparer la fluctaution des prix des biens et services dans les différents pays de l'UE  
**Last Update :** 17/01/2025 11:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/TEC00118__custom_7006313/bookmark/table?lang=en&bookmarkId=e77dcd74-2e3b-45a2-9121-553c6d328061

In [10]:
HICP_inflation = eurostat.get_data_df('tec00118')

# Renommer la colonne pour éviter les problèmes avec le backslash
HICP_inflation = HICP_inflation.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
HICP_inflation = HICP_inflation.rename(columns={"2024": "HICP_2024"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
HICP_inflation = HICP_inflation[HICP_inflation["country"].isin(code_EU["iso2"])]

#supprimer les colonnes inutiles
HICP_inflation = HICP_inflation.drop(columns=HICP_inflation.columns[0:HICP_inflation.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = HICP_inflation.columns.get_loc('country') #récupérer l'indice de la colonne country
df = HICP_inflation.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
HICP_inflation['mean_HICP_inflation'] = df.mean(axis=1) #calcul la moyenne des colonnes
HICP_inflation = HICP_inflation.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
HICP_inflation.head()

,country,mean_HICP_inflation
1,AT,2.883333
2,BE,2.525000
3,BG,2.575000
5,CY,1.241667
6,CZ,3.816667


#### General government deficit/surplus

**Variable :** GOV_deficit   
**Source :** Eurostat  
**Description :** Mesure du déficit des pays en poucentage du pays afin de mesurer la santé financière in fine la capacité à investir   
**Last Update :** 29/10/2024 11:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/TEC00127__custom_7006381/bookmark/table?lang=en&bookmarkId=f30ff9d0-3951-4e45-9add-a5cc5022402c

In [11]:
GOV_deficit = eurostat.get_data_df('tec00127')

# Renommer la colonne pour éviter les problèmes avec le backslash
GOV_deficit = GOV_deficit.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
GOV_deficit = GOV_deficit.rename(columns={"2023": "DEFICIT_2023"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GOV_deficit = GOV_deficit[GOV_deficit["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
GOV_deficit = GOV_deficit[GOV_deficit["unit"]=='PC_GDP']

#supprimer les colonnes inutiles
GOV_deficit = GOV_deficit.drop(columns=GOV_deficit.columns[0:GOV_deficit.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GOV_deficit.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GOV_deficit.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GOV_deficit['mean_GOV_deficit'] = df.mean(axis=1) #calcul la moyenne des colonnes
GOV_deficit = GOV_deficit.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GOV_deficit.head()

,country,mean_GOV_deficit
30,AT,-2.333333
31,BE,-3.466667
32,BG,-1.300000
33,CY,-2.733333
34,CZ,-1.841667


#### Energy imports dependency

**Variable :** NRJ_dependency   
**Source :** Eurostat  
**Description :** Mesure de la dépendance énergétique des pays à l'importation. Mesure le risque d'un manque de souveraineté énergétique. Indice moyenné faisant intervenir tous les secteurs de l'économie  
**Last Update :** 19/12/2024 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_7006438/bookmark/table?lang=en&bookmarkId=05cad4a2-16a0-40af-9a92-2daa73191563

In [12]:
NRJ_dependency = eurostat.get_data_df('nrg_ind_id')

# Renommer la colonne pour éviter les problèmes avec le backslash
NRJ_dependency = NRJ_dependency.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
NRJ_dependency = NRJ_dependency.rename(columns={"2023": "Dep_NRJ_2023"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
NRJ_dependency = NRJ_dependency[NRJ_dependency["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
NRJ_dependency = NRJ_dependency[NRJ_dependency["siec"]=='TOTAL']

#supprimer les colonnes inutiles
NRJ_dependency = NRJ_dependency.drop(columns=NRJ_dependency.columns[0:NRJ_dependency.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = NRJ_dependency.columns.get_loc('country') #récupérer l'indice de la colonne country
df = NRJ_dependency.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
NRJ_dependency['mean_NRJ_dependency'] = df.mean(axis=1) #calcul la moyenne des colonnes
NRJ_dependency = NRJ_dependency.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
NRJ_dependency.head()

,country,mean_NRJ_dependency
505,AT,66.289353
507,BE,78.216324
508,BG,46.046500
509,CY,96.887500
510,CZ,27.591353


#### Net greenhouse gas emissions

**Variable :** CO2_emissions   
**Source :** Eurostat  
**Description :** Mesure dle niveau d'émissions de gaz à effet de serre en équivalent CO2 sur la totalité des secteurs de l'économie   
**Last Update :** 09/01/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/SDG_13_10__custom_7006397/bookmark/table?lang=en&bookmarkId=bd680cb0-56b1-46b8-9d0a-0ba609aee133

In [13]:
CO2_emissions = eurostat.get_data_df('sdg_13_10')

# Renommer la colonne pour éviter les problèmes avec le backslash
CO2_emissions = CO2_emissions.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
CO2_emissions = CO2_emissions.rename(columns={"2022": "EmCO2_2022"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
CO2_emissions = CO2_emissions[CO2_emissions["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
CO2_emissions = CO2_emissions[CO2_emissions["src_crf"]=='TOTXMEMONIA']
CO2_emissions = CO2_emissions[CO2_emissions["unit"]=='I90']

#supprimer les colonnes inutiles
CO2_emissions = CO2_emissions.drop(columns=CO2_emissions.columns[0:CO2_emissions.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = CO2_emissions.columns.get_loc('country') #récupérer l'indice de la colonne country
df = CO2_emissions.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
CO2_emissions['mean_CO2_emissions'] = df.mean(axis=1) #calcul la moyenne des colonnes
CO2_emissions = CO2_emissions.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
CO2_emissions.head()

,country,mean_CO2_emissions
62,AT,105.006061
63,BE,94.218182
64,BG,62.218182
66,CY,144.924242
67,CZ,72.954545


#### Long term government bond yields

**Variable :** GOV_bound   
**Source :** Eurostat  
**Description :** Mesure la capacité des pays à pouvoir s'endetter sur le long terme. Calcul d'un taux d'intérêt moyen   
**Last Update :** 15/01/2025 11:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/teimf050/default/table?lang=en&category=t_irt

In [14]:
GOV_bound = eurostat.get_data_df('teimf050')

# Renommer la colonne pour éviter les problèmes avec le backslash
GOV_bound = GOV_bound.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GOV_bound = GOV_bound[GOV_bound["country"].isin(code_EU["iso2"])]

#supprimer les colonnes inutiles
GOV_bound = GOV_bound.drop(columns=GOV_bound.columns[0:GOV_bound.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GOV_bound.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GOV_bound.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GOV_bound['mean_GOV_bound'] = df.mean(axis=1) #calcul la moyenne des colonnes
GOV_bound = GOV_bound.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GOV_bound.head()

,country,mean_GOV_bound
0,AT,2.855000
1,BE,2.949167
2,BG,3.930000
3,CY,3.113333
4,CZ,4.005000


#### EU direct investments indicators in % of GDP, impact indicators and rate of return on direct investment (BPM6)

**Variable :** FDI_impact   
**Source :** Eurostat  
**Description :** Taux de retour sur investissement pour un Investisssment Direct de l'Etranger (IDE) dans l'économie nationale d'un pays    
**Last Update :** 15/01/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/bop_fdi6_ind/default/table?lang=en&category=bop_6.bop_fdi6

In [15]:
FDI_impact = eurostat.get_data_df('bop_fdi6_ind')

# Renommer la colonne pour éviter les problèmes avec le backslash
FDI_impact = FDI_impact.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
FDI_impact = FDI_impact[FDI_impact["country"].isin(code_EU["iso2"])]

"""
indic_bp :
[FGDP] GDP (flows in % of GDP)
[SGDP] GDP (stocks in % of GDP)
[IPOS] Rate of return on direct investment (FDI income/FDI stocks)
[PCPOS] FDI stocks (in % of extra EU FDI stocks)
[TIFDI] Market integration - foreign direct investment intensity - average value of inward and outward foreign direct investment flows divided by GDP, multiplied by 100

fdi_item :
[DO__D__F] Direct investment abroad (DIA)
[DI__D__F] Direct investment in the reporting economy (DIRE)
[NAP] Not applicable

"""

#Garder uniquement les unités de mesure d'intérêt
FDI_impact = FDI_impact[FDI_impact['indic_bp']=='IPOS']
FDI_impact = FDI_impact[FDI_impact["fdi_item"]=='DO__D__F']

# Récupérer la dernière valeur
first_col_value = FDI_impact.columns.get_loc('country') + 1 #récupérer l'indeice de la première colonne avce des valeurs
last_col = FDI_impact.shape[1]


FDI_impact = pd.pivot_table(
    FDI_impact,
    values=FDI_impact.columns[first_col_value:last_col],
    index=['country'],
    columns=['indic_bp', 'fdi_item'],
    aggfunc='mean')

FDI_impact.head(100)

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
indic_bp,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS,IPOS
fdi_item,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F,DO__D__F
country,,,,,,,,,,,
AT,8.592308,5.276923,5.623077,6.623077,5.554545,-4.308333,11.072727,5.390000,7.950000,13.327273,6.230000
BE,5.561538,4.215385,1.980000,50.484615,8.261538,16.507692,9.061538,8.241667,10.750000,5.333333,4.625000
BG,-0.415385,1.146154,2.700000,1.869231,2.238462,-2.107692,7.469231,0.250000,8.250000,3.216667,1.541667
CY,4.650000,4.112500,5.983333,2.300000,2.850000,5.500000,5.380000,4.300000,4.385714,3.550000,4.500000
CZ,-10.923077,-14.508333,-38.411111,-62.472727,-328.740000,-10.315385,8.081818,21.941667,21.333333,17.045455,-11.663636
DE,8.384615,9.330769,8.007692,8.033333,9.930769,8.691667,9.515385,7.338462,7.850000,10.583333,10.858333
DK,10.884615,10.523077,10.969231,7.423077,7.630769,7.192308,11.146154,11.161538,13.733333,9.625000,11.008333


In [16]:
#Supprimer le multi-index

# Réinitialiser l'index pour récupérer les pays
FDI_impact = FDI_impact.reset_index()

# Vérifier si les colonnes ont un MultiIndex
if isinstance(FDI_impact.columns, pd.MultiIndex):
    # Aplatir les colonnes MultiIndex proprement
    FDI_impact.columns = ['_'.join(filter(None, map(str, col))) for col in FDI_impact.columns]

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = FDI_impact.columns.get_loc('country') #récupérer l'indice de la colonne country
df = FDI_impact.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
FDI_impact['mean_FDI_impact'] = df.mean(axis=1) #calcul la moyenne des colonnes
FDI_impact = FDI_impact.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

FDI_impact.head()


,country,mean_FDI_impact
0,AT,6.484691
1,BE,11.365664
2,BG,2.378030
3,CY,4.319232
4,CZ,-37.148363


#### Gross domestic product (GDP) and main components (output, expenditure and income)

**Variable :** GOV_GDP   
**Source :** Eurostat  
**Description :** Représente les dépense d'investissments en poucentage de PIB   
**Last Update :** 29/01/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/nama_10_gdp/default/table?lang=en&category=na10.nama10.nama_10_ma

In [17]:
GOV_GDP = eurostat.get_data_df('nama_10_gdp')

# Renommer la colonne pour éviter les problèmes avec le backslash
GOV_GDP = GOV_GDP.rename(columns={"geo\TIME_PERIOD": "country"})

# Renommer la colonne pour éviter les problèmes de colusions lors du merge
GOV_GDP = GOV_GDP.rename(columns={"2023": "PIB_2023"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GOV_GDP = GOV_GDP[GOV_GDP["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
GOV_GDP = GOV_GDP[GOV_GDP['na_item']=='B1G']
GOV_GDP = GOV_GDP[GOV_GDP["unit"]=='PC_GDP']

#supprimer les colonnes inutiles
GOV_GDP = GOV_GDP.drop(columns=GOV_GDP.columns[0:GOV_GDP.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GOV_GDP.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GOV_GDP.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GOV_GDP['mean_GOV_GDP'] = df.mean(axis=1) #calcul la moyenne des colonnes
GOV_GDP = GOV_GDP.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GOV_GDP.head()

,country,mean_GOV_GDP
20380,AT,89.144828
20382,BE,89.534483
20383,BG,87.341379
20385,CY,88.927586
20386,CZ,90.734483


#### Green bond issuance by corporates and governments

**Variable :** GOV_GDP   
**Source :** European Environment Agency (EEA)   
**Description :** Pourcentage d'obligations vertes émises par les entreprises et les gouvernements parmi les obligations totales émises  
**Last Update :** 11/06/2024 23:00   
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_13_70a/default/table?lang=en&category=sdg.sdg_13

In [18]:
GREEN_Bound = eurostat.get_data_df('sdg_13_70a')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_Bound = GREEN_Bound.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_Bound = GREEN_Bound[GREEN_Bound["country"].isin(code_EU["iso2"])]

#supprimer les colonnes inutiles
GREEN_Bound = GREEN_Bound.drop(columns=GREEN_Bound.columns[0:GREEN_Bound.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_Bound.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_Bound.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_Bound['mean_GREEN_Bound'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_Bound = GREEN_Bound.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GREEN_Bound.head()


,country,mean_GREEN_Bound
0,AT,8.3
1,BE,14.8
2,BG,0.0
3,CY,0.0
4,CZ,3.3


#### Climate related economic losses – values at constant 2022 prices

**Variable :** GREEN_Losses   
**Source :** European Environment Agency (EEA)   
**Description :** Pertes économiques estimées dues au changement climatique.Estimées en CP_EUR_HAB en euro constant 2022  
**Last Update :** 06/11/2024 23:00   
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_13_40/default/table?lang=en&category=sdg.sdg_13

In [19]:
GREEN_Losses = eurostat.get_data_df('sdg_13_40')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_Losses = GREEN_Losses.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_Losses = GREEN_Losses[GREEN_Losses["country"].isin(code_EU["iso2"])]
"""
[VAL_A] Annual value
[AVG_30Y] Thirty-year average
[CP_MEUR] Current prices, million euro
[CP_EUR_HAB] Current prices, euro per capita
"""
#Garder uniquement les unités de mesure d'intérêt
GREEN_Losses = GREEN_Losses[GREEN_Losses['statinfo']=='VAL_A']
GREEN_Losses = GREEN_Losses[GREEN_Losses["unit"]=='CP_EUR_HAB']

#supprimer les colonnes inutiles
GREEN_Losses = GREEN_Losses.drop(columns=GREEN_Losses.columns[0:GREEN_Losses.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_Losses.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_Losses.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_Losses['mean_GREEN_Losses'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_Losses = GREEN_Losses.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GREEN_Losses.head()


,country,mean_GREEN_Losses
66,AT,40.223636
67,BE,34.092500
68,BG,16.006136
70,CY,12.567045
71,CZ,40.780682


#### Contribution to the international 100bn USD commitment on climate related expending (source: DG CLIMA, EIONET)

**Variable :** GREEN_COP21   
**Source :** Eurostat   
**Description :** Contribution des pays européens au fonds de 100 milliards de dollars institué par les Accords de Paris et devant aider à lutter contre le changement climatique
**Last Update :** sdg_13_50  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_13_50/default/table?lang=en&category=sdg.sdg_13

In [20]:
GREEN_COP21 = eurostat.get_data_df('sdg_13_50')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_COP21 = GREEN_COP21.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_COP21 = GREEN_COP21[GREEN_COP21["country"].isin(code_EU["iso2"])]

#supprimer les colonnes inutiles
GREEN_COP21 = GREEN_COP21.drop(columns=GREEN_COP21.columns[0:GREEN_COP21.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_COP21.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_COP21.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_COP21['mean_GREEN_COP21'] = df.sum(axis=1) #calcul la moyenne des colonnes
GREEN_COP21 = GREEN_COP21.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées
GREEN_COP21.head()

,country,mean_GREEN_COP21
0,AT,2641.17
1,BE,1614.33
2,BG,0.47
4,CY,0.00
5,CZ,86.71


#### Gross value added in environmental goods and services sector

**Variable :** GREEN_gross_value
**Source :** Eurostat   
**Description :** Valeur ajoutée du secteur "vert" dans les biens et services en pourcentage de PIB   
**Last Update :** 02/12/2024   
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_12_61/default/table?lang=en&category=sdg.sdg_09

In [21]:
GREEN_gross_value = eurostat.get_data_df('sdg_12_61')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_gross_value = GREEN_gross_value.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_gross_value = GREEN_gross_value[GREEN_gross_value["country"].isin(code_EU["iso2"])]

"""
[CLV10_MEUR] Chain linked volumes (2010), million euro
[PC_GDP] Percentage of gross domestic product (GDP)
"""
#Garder uniquement les unités de mesure d'intérêt
GREEN_gross_value = GREEN_gross_value[GREEN_gross_value['unit']=='PC_GDP']

#supprimer les colonnes inutiles
GREEN_gross_value = GREEN_gross_value.drop(columns=GREEN_gross_value.columns[0:GREEN_gross_value.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_gross_value.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_gross_value.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_gross_value['mean_GREEN_gross_value'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_gross_value = GREEN_gross_value.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

GREEN_gross_value.head()

,country,mean_GREEN_gross_value
33,AT,4.484000
34,BE,1.795556
35,BG,2.332000
37,CY,1.832000
38,CZ,2.486667


#### Energy productivity

**Variable :** NRJ_productivity  
**Source :** Eurostat   
**Description :** Productivité de l'énergie estimée en équivalent pétrole de 1kg et en parité de pouvoir d'achat  
**Last Update :** 19/12/2024 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_07_30/default/table?lang=en&category=sdg.sdg_07

In [22]:
NRJ_productivity = eurostat.get_data_df('sdg_07_30')

# Renommer la colonne pour éviter les problèmes avec le backslash
NRJ_productivity = NRJ_productivity.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
NRJ_productivity = NRJ_productivity[NRJ_productivity["country"].isin(code_EU["iso2"])]

"""
[EUR_KGOE] Euro per kilogram of oil equivalent (KGOE)
[PPS_KGOE] Purchasing power standard (PPS) per kilogram of oil equivalent
"""
#Garder uniquement les unités de mesure d'intérêt
NRJ_productivity = NRJ_productivity[NRJ_productivity['unit']=='PPS_KGOE']

#supprimer les colonnes inutiles
NRJ_productivity = NRJ_productivity.drop(columns=NRJ_productivity.columns[0:NRJ_productivity.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = NRJ_productivity.columns.get_loc('country') #récupérer l'indice de la colonne country
df = NRJ_productivity.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
NRJ_productivity['mean_NRJ_productivity'] = df.mean(axis=1) #calcul la moyenne des colonnes
NRJ_productivity = NRJ_productivity.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

NRJ_productivity.head()

,country,mean_NRJ_productivity
39,AT,8.568333
41,BE,5.556667
42,BG,4.720000
43,CY,7.485000
44,CZ,5.612500


#### Premature deaths due to exposure to fine particulate matter (PM2.5)

**Variable :** POLL_death  
**Source :** European Environment Agency (EEA)   
**Description :** Morts prématurées dues à une exposition aux particules fines    
**Last Update :** 03/02/2025 23:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/sdg_11_52/default/table?lang=en&category=sdg.sdg_11

In [23]:
POLL_death = eurostat.get_data_df('sdg_11_52')

# Renommer la colonne pour éviter les problèmes avec le backslash
POLL_death = POLL_death.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
POLL_death = POLL_death[POLL_death["country"].isin(code_EU["iso2"])]

"""
[NR] Number
[RT] Rate
"""
#Garder uniquement les unités de mesure d'intérêt
POLL_death = POLL_death[POLL_death['unit']=='RT']

#supprimer les colonnes inutiles
POLL_death = POLL_death.drop(columns=POLL_death.columns[0:POLL_death.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = POLL_death.columns.get_loc('country') #récupérer l'indice de la colonne country
df = POLL_death.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
POLL_death['mean_POLL_death'] = df.mean(axis=1) #calcul la moyenne des colonnes
POLL_death = POLL_death.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

POLL_death.head()

,country,mean_POLL_death
39,AT,58.470588
41,BE,62.411765
42,BG,205.941176
44,CY,88.764706
45,CZ,90.882353


#### Environmental tax revenues

**Variable :** GREEN_tax  
**Source :** Eurostat   
**Description :** Total des taxes environnementales en pourcentage du PIB
**Last Update :** 28/01/2025 11:00  
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/env_ac_tax/default/table?lang=en&category=env.env_eta

In [24]:
GREEN_tax = eurostat.get_data_df('env_ac_tax')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_tax = GREEN_tax.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_tax = GREEN_tax[GREEN_tax["country"].isin(code_EU["iso2"])]

"""
[ENV] Total environmental taxes
[NRG] Energy taxes
[TRA] Transport taxes
[POL_RES] Taxes on pollution/resources

[MIO_EUR] Million euro
[MIO_NAC] Million units of national currency
[PC_TSCO] Percentage of total revenues from taxes and social contributions (including imputed social contributions)
[PC_TSCO_X_ISCO] Percentage of total revenues from taxes and social contributions (excluding imputed social contributions)
[PC_GDP] Percentage of gross domestic product (GDP)
"""
#Garder uniquement les unités de mesure d'intérêt
GREEN_tax = GREEN_tax[GREEN_tax['tax']=='ENV']
GREEN_tax = GREEN_tax[GREEN_tax['unit']=='PC_GDP']

#supprimer les colonnes inutiles
GREEN_tax = GREEN_tax.drop(columns=GREEN_tax.columns[0:GREEN_tax.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_tax.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_tax.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_tax['mean_GREEN_tax'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_tax = GREEN_tax.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

GREEN_tax.head()

,country,mean_GREEN_tax
64,AT,2.444828
65,BE,2.485172
66,BG,2.704483
68,CY,3.049310
69,CZ,2.159655


#### Employment in the environmental goods and services sector

**Variable :** GREEN_employment  
**Source :** Eurostat   
**Description :** Part totale des emplois "verts" sur les emplois du secteur des biens et services   
**Last Update :** 30/04/2024 23:00    
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/env_ac_egss1/default/table?lang=en&category=env.env_egs

In [25]:
GREEN_employment = eurostat.get_data_df('env_ac_egss1')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_employment = GREEN_employment.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_employment = GREEN_employment[GREEN_employment["country"].isin(code_EU["iso2"])]

"""
[TOT_EGSS] Total environmental goods and services sector
[ANC] Ancillary activities
[MKT] Market activities
[ES_CS] Market activities: environmental specific and connected services
[C_REP] Market activities: cleaner and resource efficient products
[NMKT] Non-market activities
[OWN_USE] Own final use activities
"""
#Garder uniquement les unités de mesure d'intérêt
GREEN_employment = GREEN_employment[GREEN_employment['ty']=='TOT_EGSS']

GREEN_employment = GREEN_employment.groupby('country').sum()
GREEN_employment.reset_index(inplace=True)

#supprimer les colonnes inutiles
GREEN_employment = GREEN_employment.drop(columns=GREEN_employment.columns[1:GREEN_employment.columns.get_loc('unit')+1])

# Faire la moyenne des colonnes pour une valeur annuelle
df = GREEN_employment.iloc[:, 1:] #Retourne la table avec uniquement les prix
GREEN_employment['mean_GREEN_employment'] = df.sum(axis=1) #calcul la moyenne des colonnes
GREEN_employment = GREEN_employment.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

GREEN_employment

,country,mean_GREEN_employment
0,AT,19383640.0
1,BE,5045574.0
2,BG,3773866.0
3,CY,372371.0
4,CZ,7642963.0
5,DE,38082271.0
6,DK,6043897.0
7,EE,2096800.0
8,ES,31114862.0
9,FI,10686157.0


#### Investments in climate change mitigation by NACE Rev. 2 activity

**Variable :** GREEN_investments  
**Source :** Eurostat   
**Description :** Investissemnts totaux en millions d'euros 2020 pour la lutte contre le déréglement climatique  
**Last Update :** 25/11/2024 23:00    
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/env_ac_ccminv/default/table?lang=en&category=env.env_egs

In [26]:
GREEN_investments = eurostat.get_data_df('env_ac_ccminv')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_investments = GREEN_investments.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_investments = GREEN_investments[GREEN_investments["country"].isin(code_EU["iso2"])]

"""
[TOTAL] Total - all NACE activities
[A] Agriculture, forestry and fishing
[B] Mining and quarrying
[C] Manufacturing
[D] Electricity, gas, steam and air conditioning supply

[MIO_EUR] Million euro
[PC_GDP] Percentage of gross domestic product (GDP)
[CLV20_MEUR] Chain linked volumes (2020), million euro
"""
#Garder uniquement les unités de mesure d'intérêt
GREEN_investments = GREEN_investments[GREEN_investments['nace_r2']=='TOTAL']
GREEN_investments = GREEN_investments[GREEN_investments['unit']=='CLV20_MEUR']

#supprimer les colonnes inutiles
GREEN_investments = GREEN_investments.drop(columns=GREEN_investments.columns[0:GREEN_investments.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_investments.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_investments.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_investments['mean_GREEN_investments'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_investments = GREEN_investments.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

GREEN_investments.head()

,country,mean_GREEN_investments
857,AT,3213.635789
858,BE,3966.281053
859,BG,450.814737
860,CY,26.490000
861,CZ,1208.926316


#### Environmental protection investments of total economy

**Variable :** GREEN_enviro_protection  
**Source :** Eurostat   
**Description :** Investissemnts en pourcentage du PIB dans la protection de l'environnement
**Last Update :** 06/12/2024 23:00    
**URL :**  
https://ec.europa.eu/eurostat/databrowser/view/env_ac_epite1/default/table?lang=en

In [27]:
GREEN_enviro_protection = eurostat.get_data_df('env_ac_epite1')

# Renommer la colonne pour éviter les problèmes avec le backslash
GREEN_enviro_protection = GREEN_enviro_protection.rename(columns={"geo\TIME_PERIOD": "country"})

# Filtrer les lignes où "geo_TIME_PERIOD" est dans la colonne "iso2" de code_EU
GREEN_enviro_protection = GREEN_enviro_protection[GREEN_enviro_protection["country"].isin(code_EU["iso2"])]

#Garder uniquement les unités de mesure d'intérêt
GREEN_enviro_protection = GREEN_enviro_protection[GREEN_enviro_protection['unit']=='PC_GDP']

#supprimer les colonnes inutiles
GREEN_enviro_protection = GREEN_enviro_protection.drop(columns=GREEN_enviro_protection.columns[0:GREEN_enviro_protection.columns.get_loc('country')])

# Faire la moyenne des colonnes pour une valeur annuelle
column_index = GREEN_enviro_protection.columns.get_loc('country') #récupérer l'indice de la colonne country
df = GREEN_enviro_protection.iloc[:, (column_index+1):] #Retourne la table avec uniquement les prix
GREEN_enviro_protection['mean_GREEN_enviro_protection'] = df.mean(axis=1) #calcul la moyenne des colonnes
GREEN_enviro_protection = GREEN_enviro_protection.iloc[:, [0, -1]] #supprime les valeurs semestrielles déjà moyennées

GREEN_enviro_protection.head(100)

,country,mean_GREEN_enviro_protection
28,AT,0.325000
29,BE,0.462500
30,BG,0.862500
32,CY,0.418182
33,CZ,0.750000
34,DE,0.400000
35,DK,0.437500
36,EE,0.650000
38,ES,0.246154
40,FI,0.260000


In [30]:
# Liste des DataFrames à fusionner
dataframes = [
    electricity_price, GDP_per_capita, HICP_inflation, NRJ_dependency, NRJ_productivity, POLL_death,
    GREEN_Bound, GREEN_COP21, GREEN_Losses, GREEN_employment, GREEN_enviro_protection, GREEN_gross_value,
    GREEN_investments, GREEN_tax, FDI_impact, GOV_GDP, GOV_bound, GOV_deficit, share_renew_energy, CO2_emissions
]

# Fusionner les DataFrames successivement
data = code_EU.rename(columns={'iso2': 'country'})

for df in dataframes:
    data = pd.merge(data, df, on='country', how='left')

data.dropna(inplace=True) #supprimer la Grèce

# Afficher un aperçu des données fusionnées
data

,name_en,name_fr,country,iso3,mean_energy_price,mean_GDP_per_capita,mean_HICP_inflation,mean_NRJ_dependency,mean_NRJ_productivity,mean_POLL_death,mean_GREEN_Bound,mean_GREEN_COP21,mean_GREEN_Losses,mean_GREEN_employment,mean_GREEN_enviro_protection,mean_GREEN_gross_value,mean_GREEN_investments,mean_GREEN_tax,mean_FDI_impact,mean_GOV_GDP,mean_GOV_bound,mean_GOV_deficit,mean_share_renew_energy,mean_CO2_emissions
0,Croatia,Croatie,HR,HRV,0.175217,2.945833,2.591667,47.643618,7.673333,114.941176,0.0,0.47,22.510682,2319895.0,0.562500,1.644444,375.637368,3.682759,-6.537475,83.389655,3.284167,-2.541667,26.50435,79.372727
1,Romania,Roumanie,RO,ROU,0.287683,4.429167,3.841667,26.227059,8.495833,126.117647,0.7,52.97,21.244773,19107921.0,0.558824,2.802000,2227.852105,2.294828,12.117424,90.237931,6.394167,-4.091667,22.44690,49.530303
2,Latvia,Lettonie,LV,LVA,0.289067,4.579167,3.341667,58.673618,6.618750,73.411765,10.5,0.82,13.584318,1640729.0,0.400000,2.837778,458.715789,2.701034,6.196840,88.148276,3.273333,-2.208333,36.54865,43.642424
3,Austria,Autriche,AT,AUT,0.198467,0.916667,2.883333,66.289353,8.568333,58.470588,8.3,2641.17,40.223636,19383640.0,0.325000,4.484000,3213.635789,2.444828,6.484691,89.144828,2.855000,-2.333333,31.83285,105.006061
4,Lithuania,Lituanie,LT,LTU,0.247333,4.891667,3.666667,63.143147,6.910417,76.235294,0.0,18.99,16.253864,3705280.0,0.691667,2.415385,666.511053,2.072414,0.894307,89.500000,2.880000,-1.308333,22.81555,39.812121
5,Spain,Espagne,ES,ESP,0.252800,1.012000,1.850000,73.347853,8.361250,44.058824,7.6,6967.47,52.181136,31114862.0,0.246154,2.469000,4896.697895,1.860690,8.106235,91.203333,3.149167,-5.666667,15.26090,121.993939
6,Slovenia,Slovénie,SI,SVN,0.165267,2.124000,2.225000,49.982029,6.699583,74.411765,0.0,42.87,191.309091,2391372.0,0.914286,1.606923,355.560000,3.664138,7.977310,87.053333,3.112500,-3.408333,21.74110,98.715152
7,Belgium,Belgique,BE,BEL,0.257600,1.100000,2.525000,78.216324,5.556667,62.411765,14.8,1614.33,34.092500,5045574.0,0.462500,1.795556,3966.281053,2.485172,11.365664,89.534483,2.949167,-3.466667,7.67355,94.218182
8,Netherlands,Pays-Bas,NL,NLD,0.250000,1.116000,2.633333,37.543235,6.666667,50.941176,11.1,6742.48,15.590000,19285852.0,0.477778,2.131818,2641.880000,3.531034,5.896115,89.556667,2.640000,-1.008333,6.78340,96.000000
9,Poland,Pologne,PL,POL,0.167360,3.729167,3.466667,21.813794,6.564583,114.058824,3.8,147.37,12.435682,15233320.0,0.600000,2.348889,2674.091579,2.471724,10.176542,88.282759,5.583333,-3.033333,11.36660,84.763636


In [32]:
from google.colab import files

data.to_csv("EU_energy_eco_data.csv", index=False)
files.download("EU_energy_eco_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>